In [1]:
import pandas as pd
import requests
import re
from unidecode import unidecode

In [2]:
import understatapi
client = understatapi.UnderstatClient()

In [12]:
from sqlalchemy import create_engine
import psycopg2

# Database connection settings (replace with your credentials)
db_config = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'ishdeep',
    'host': 'localhost',
    'port': 5432,
}

# Create a connection engine
engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")

def save_to_postgres(dataframe, table_name):
    try:
        # Append data to the PostgreSQL table
        dataframe.to_sql(table_name, engine, if_exists='append', index=False)
        print(f"Data successfully saved to {table_name} table.")
    except Exception as e:
        print(f"Error saving data to PostgreSQL: {e}")


In [4]:
## EPL , Bundesliga , Serie_A , Ligue_1 , La_Liga
league = "Ligue_1"
seasons = ["2014","2015","2016","2017","2018","2019","2020","2021","2022","2023"]

In [5]:
matches = []

for season in seasons:
    league_data = client.league(league=league).get_match_data(season=season)
    
    # Check if the league_data is a list
    if isinstance(league_data, list):
        for match in league_data:
            if isinstance(match, dict):  # Ensure each match is a dictionary
                match_data = {
                    'id': match.get('id'),
                    'home_team': match['h'].get('title') if isinstance(match.get('h'), dict) else None,
                    'away_team': match['a'].get('title') if isinstance(match.get('a'), dict) else None,
                    'home_goals': match['goals'].get('h') if isinstance(match.get('goals'), dict) else None,
                    'away_goals': match['goals'].get('a') if isinstance(match.get('goals'), dict) else None,
                    'home_xG': match['xG'].get('h') if isinstance(match.get('xG'), dict) else None,
                    'away_xG': match['xG'].get('a') if isinstance(match.get('xG'), dict) else None,
                    'datetime': match.get('datetime')
                }
                matches.append(match_data)
            else:
                print(f"Unexpected match data: {match}")
    else:
        print(f"Unexpected league data structure for season {season}: {league_data}")

# Convert the list of dictionaries into a DataFrame
matches_df = pd.DataFrame(matches)


In [6]:
matches_df.tail()

,id,home_team,away_team,home_goals,away_goals,home_xG,away_xG,datetime
3721,23672,Reims,Rennes,2,1,1.39024,1.25648,2024-05-19 19:00:00
3722,23673,Lille,Nice,2,2,1.59935,1.32909,2024-05-19 19:00:00
3723,23674,Lorient,Clermont Foot,5,0,4.92005,0.737223,2024-05-19 19:00:00
3724,23675,Monaco,Nantes,4,0,1.85296,0.885758,2024-05-19 19:00:00
3725,23676,Toulouse,Brest,0,3,0.984204,1.02904,2024-05-19 19:00:00


In [7]:
all_shot_data = []

import datetime

current_timestamp = datetime.datetime.now().timestamp()

for index,row in matches_df.iterrows():
    match_datetime = pd.to_datetime(row['datetime']).timestamp()
    
    if match_datetime <= current_timestamp:
        #print(row['id'])
        try:
            shot_data = client.match(match=row['id']).get_shot_data()
        except Exception as e:
            print(e)
        all_shot_data.append(shot_data)

4238 is not a valid match
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
12990 is not a valid match
12998 is not a valid match
12991 is not a valid match
12992 is not a valid match
12993 is not a valid match
12994 is not a valid match
12997 is not a valid match
12995 is not a valid match
12996 is not a valid match
12989 is not a valid match
12980 is not a valid match
13001 is not a valid match
13007 is not a valid match
12999 is not a valid match
13000 is not a valid match
13005 is not a valid match
13006 is not a valid match
13008 is not a valid match
13004 is not a valid match
13002 is not a valid match
13003 is not a valid match
13009 is not a valid match
13010 is not a valid match
13011 is not a valid match
13012 is not a valid match
13013 is not a valid match
13014 is not a valid match
13015 is not a valid match
13016 is not a valid match
13017 is not a valid match
13018 is not a valid match
13019 is not a valid match
13020 is not a va

In [8]:
# Initialize an empty list to hold individual shot records
compiled_shot_data = []

# Loop through each match in `all_shot_data`
for match in all_shot_data:
    # Get home shots and away shots from the match
    home_shots = match.get('h', [])
    away_shots = match.get('a', [])
    
    # Add a column to indicate if it's a home or away shot, then extend our list
    for shot in home_shots:
        shot['h_a'] = 'h'  # Indicate as home shot
        compiled_shot_data.append(shot)
    
    for shot in away_shots:
        shot['h_a'] = 'a'  # Indicate as away shot
        compiled_shot_data.append(shot)

# Convert the list of shot records to a DataFrame
compiled_shot_df = pd.DataFrame(compiled_shot_data)

In [9]:
compiled_shot_df['league'] = league

In [10]:
compiled_shot_df = compiled_shot_df.drop_duplicates()

In [13]:
#compiled_shot_df.to_csv(f'C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/Data/{league}/2024-25/shot_data.csv')
try:
    save_to_postgres(compiled_shot_df, 'understat_shots_tb')
except Exception as e:
    print(e)

Data successfully saved to understat_shots_tb table.
